# DA for AQMS in QLD
### The current version is to wrangle and clean the AQ data from the AQMS in QLD, or AMNS in QLD

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [76]:
# the location of the data
data_folder = 'C:\\Users\\CHENGG\\Desktop\\IA317900\\data\\'
data_BC, data_MP, data_SG = data_folder + 'BoatCreek\\', data_folder + 'MomerialPark\\', data_folder + 'SouthGladstone\\'
file_BC = 'air-quality-data-boa-2024-10-28-to-2024-11-04.csv'
file_MP = 'air-quality-data-cgm-2024-10-28-to-2024-11-04.csv'
file_SG = 'air-quality-data-qal-2024-10-28-to-2024-11-04.csv'

In [94]:
def qld_amns_wrangler(data):
    df = pd.read_csv(data)
    print(f'Here are the items that the station is monitoring : {df.Parameter.unique()}')
    # wrangle the data
    df['Time'] = pd.to_datetime(df['Datetime (UTC)']).dt.tz_convert('Australia/Brisbane')
    df['year'] = df.Time.dt.year
    df['month'] = df.Time.dt.month
    df['day'] = df.Time.dt.day
    df['hour'] = df.Time.dt.hour
    # need to covert the unit for NO2 from ppm to ug/m3
    try:
        df.loc[df['Parameter'] == 'Nitrogen dioxide', 'Measurement'] = df.loc[df['Parameter'] == 'Nitrogen dioxide', 'Measurement']*1880
        df.loc[df['Parameter'] == 'Nitrogen dioxide', 'Measurement units'] = 'µg/m³'
        print('NOx unit has been changed from ppm to µg/m³')
    except:
        pass
    return df

In [95]:
# load the data
df_bc = qld_amns_wrangler(data_BC+file_BC)
df_mp = qld_amns_wrangler(data_MP+file_MP)
df_sg = qld_amns_wrangler(data_SG+file_SG)

Here are the items that the station is monitoring : ['Nitrogen dioxide' 'Sulfur dioxide' 'Particle PM10' 'Particle PM2.5'
 'Visibility' 'Wind direction' 'Wind speed' 'Temperature']
NOx unit has been changed from ppm to µg/m³
Here are the items that the station is monitoring : ['Nitrogen dioxide' 'Ozone' 'Sulfur dioxide' 'Benzene' 'Toluene' 'Xylene'
 'Formaldehyde']
NOx unit has been changed from ppm to µg/m³
Here are the items that the station is monitoring : ['Nitrogen dioxide' 'Sulfur dioxide' 'Particle PM10' 'Particle PM2.5'
 'Visibility' 'Wind direction' 'Wind speed' 'Humidity' 'Temperature'
 'Solar radiation' 'Rainfall']
NOx unit has been changed from ppm to µg/m³


In [96]:
df_sg.head(11)

,Station,Datetime (UTC),Parameter,Measurement,Measurement units,Measurement running average,Measurement running average units,Validated,Time,year,month,day,hour
0,South Gladstone,2024-11-04T03:00:00.000Z,Nitrogen dioxide,3.76,µg/m³,0.002,ppm (1hr avg),N,2024-11-04 13:00:00+10:00,2024,11,4,13
1,South Gladstone,2024-11-04T02:00:00.000Z,Nitrogen dioxide,3.76,µg/m³,0.002,ppm (1hr avg),N,2024-11-04 12:00:00+10:00,2024,11,4,12
2,South Gladstone,2024-11-04T01:00:00.000Z,Nitrogen dioxide,1.88,µg/m³,0.001,ppm (1hr avg),N,2024-11-04 11:00:00+10:00,2024,11,4,11
3,South Gladstone,2024-11-04T00:00:00.000Z,Nitrogen dioxide,3.76,µg/m³,0.002,ppm (1hr avg),N,2024-11-04 10:00:00+10:00,2024,11,4,10
4,South Gladstone,2024-11-03T23:00:00.000Z,Nitrogen dioxide,5.64,µg/m³,0.003,ppm (1hr avg),N,2024-11-04 09:00:00+10:00,2024,11,4,9
5,South Gladstone,2024-11-03T22:00:00.000Z,Nitrogen dioxide,9.40,µg/m³,0.005,ppm (1hr avg),N,2024-11-04 08:00:00+10:00,2024,11,4,8
6,South Gladstone,2024-11-03T21:00:00.000Z,Nitrogen dioxide,15.04,µg/m³,0.008,ppm (1hr avg),N,2024-11-04 07:00:00+10:00,2024,11,4,7
7,South Gladstone,2024-11-03T20:00:00.000Z,Nitrogen dioxide,20.68,µg/m³,0.011,ppm (1hr avg),N,2024-11-04 06:00:00+10:00,2024,11,4,6
8,South Gladstone,2024-11-03T19:00:00.000Z,Nitrogen dioxide,5.64,µg/m³,0.003,ppm (1hr avg),N,2024-11-04 05:00:00+10:00,2024,11,4,5
9,South Gladstone,2024-11-03T18:00:00.000Z,Nitrogen dioxide,1.88,µg/m³,0.001,ppm (1hr avg),N,2024-11-04 04:00:00+10:00,2024,11,4,4
